In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
master = webdriver.Chrome('../chromedriver') #Master page
detail = webdriver.Chrome('../chromedriver') #Page detail by each sitter
import re
import pandas as pd
import numpy as np
from random import randrange
from time import sleep
from datetime import datetime as dt

/var/folders/79/4p0_7kz92s7ddc2_l55m77mw0000gn/T/ipykernel_1482/2665384660.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  master = webdriver.Chrome('../chromedriver') #Master page
/var/folders/79/4p0_7kz92s7ddc2_l55m77mw0000gn/T/ipykernel_1482/2665384660.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  detail = webdriver.Chrome('../chromedriver') #Page detail by each sitter


In [2]:
# Click the chosen object.
def click_object(driver, obj):
    try: 
        driver.execute_script("arguments[0].click();", obj )
    except:
        print ('Click error!')

In [3]:
# Sleep random (from f sec to t sec) to make sure the page finish loading.
def sleep_random(f = 5, t = 30):
    sleep(randrange(f,t))

In [4]:
# Zoom out the map to maximum level to load all sitters around that area.
def zoom_out_map(driver):
    try:
        #wait until zoom out button available
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="base-content"]/div[1]/div[3]/div[2]/div/div/div/div[8]/div/div/div/button[2]')))   
        zoom_out = master.find_element(By.XPATH,'//*[@id="base-content"]/div[1]/div[3]/div[2]/div/div/div/div[8]/div/div/div/button[2]')
        for i in range(10): #zoom out to max level
            sleep_random(1,2)
            zoom_out.click()    
    except:
        print ('Map do not appear')

In [5]:
# Get the SitterID from the link of avatar picture.
def get_sitterID(sitter):
    try:
        src_url = sitter.img['src']
        sitterID = re.findall('people\/\w*', src_url)[0].replace('people/','')
        return sitterID
    except:
        return None

In [6]:
# Parse sitter page to get all information need.
def parse_sitter(driver, sitter):
## table sitter_info
# - sitterID
# - sitterName
# - numReviews
# - rate
# - repeatClients
# - responseRate 
# - updateToClient #Clients received photo updates
# - yearsExperience
# - Qualify
# - trainFirstAid
# - srvBoarding #in the sitter's home
# - srvHouseSitting
# - srvDropIn
# - srvDogWalking
# - srvDoggyDayCare
# - long
# - lat
# - image
# - url

## table sitter_ext
# - sitterID
# - availableHost
# - availableHome
    sleep_random(3,10)
    sitter_new = sitter
    soupD = BeautifulSoup(driver.page_source, 'lxml')
# Check if the site loaded is correct sitterID
    if sitter_new['sitterID'][0] == get_sitterID(soupD.find('button', class_='col-md-4 text-center provider-thumb-button js-shared-gallery-trigger')):    
# - numReviews
        try:
            numReviews = int(soupD.find('div', {'data-qa-id':'member-profile-reviews-widget-reviews'}).text.replace(' Reviews',''))
        except:
            numReviews = None
        sitter_new['numReviews'] = [numReviews]

# - repeatClients
        try:
            repeatClients=int(soupD.select_one('#base-content > div:nth-child(4) > div > div > div.col-md-8 > section.reviews.profile-section > div:nth-child(3) > div.section-col-meta.text-center-xs > div.rover-section-widget.reviews-widget.has-icon.text-left > div:nth-child(2)').strong.text )
        except:
            repeatClients = None
        
        sitter_new['repeatClients'] = [repeatClients]
# - responseRate 
        try:
            responseRate=1
        except:
            responseRate=1
        sitter_new['repeatClients'] = [repeatClients]
# - updateToClient #Clients received photo updates
        try:
            updateToClient=1
        except:
            updateToClient=1
        sitter_new['updateToClient'] = [updateToClient]
# - yearsExperience
        try:
            yearsExperience=int(soupD.select_one('#base-content > div:nth-child(4) > div > div > div.col-md-8 > section.about.profile-section > div:nth-child(3) > div.section-col-meta.text-center-xs > div.rover-section-widget.has-icon > div:nth-child(2)').strong.text )
        except:
            yearsExperience = None
        sitter_new['yearsExperience'] = [yearsExperience]
# - qualify
        try:
            qualify='Basic'
        except:
            qualify=None
        sitter_new['qualify'] = [qualify]
# - trainFirstAid
        try:
            trainFirstAid = None
        except:
            trainFirstAid = None
        sitter_new['trainFirstAid'] = [trainFirstAid]
# - srvBoarding #in the sitter's home
        try:
            srvBoarding = int(soupD.find('span', {'data-qa-id':'member-pages-rates-overnight-boarding'}).text.replace('\n','').strip().replace('$',''))
        except:
            srvBoarding = None
        sitter_new['srvBoarding'] = [srvBoarding]
# - srvHouseSitting    
        try:
            srvHouseSitting = int(soupD.find('span', {'data-qa-id':'member-pages-rates-overnight-traveling'}).text.replace('\n','').strip().replace('$',''))
        except:
            srvHouseSitting = None
        sitter_new['srvHouseSitting'] = [srvHouseSitting]
# - srvDropIn
        try:
            srvDropIn = int(soupD.find('span', {'data-qa-id':'member-pages-rates-drop-in'}).text.replace('\n','').strip().replace('$',''))
        except:
            srvDropIn = None
        sitter_new['srvDropIn'] = [srvDropIn]
# - srvDogWalking
        try:
            srvDogWalking = int(soupD.find('span', {'data-qa-id':'member-pages-rates-dog-walking'}).text.replace('\n','').strip().replace('$',''))
        except:
            srvDogWalking = None
        sitter_new['srvDogWalking'] = [srvDogWalking]
# - srvDoggyDayCare
        try:
            srvDoggyDayCare = int(soupD.find('span', {'data-qa-id':'member-pages-rates-doggy-day-care'}).text.replace('\n','').strip().replace('$',''))
        except:
            srvDoggyDayCare = None
        sitter_new['srvDoggyDayCare'] = [srvDoggyDayCare]
# - lat
# - long

        try:
            tmp = soupD.find('div', class_='member-profile-map').img['data-src'] # get Maps link & center point
            tmp = tmp[tmp.find('center')+7: tmp.find('&channel')]
            lat = float(tmp.split('%2C')[0])
            long = float(tmp.split('%2C')[1])
        except:
            lat = 43.65
            long = -79.41 # Address of Service of Ontario - Toronto
        sitter_new['lat'] = [lat]
        sitter_new['long'] = [long]
# - image
        try:
            image=soupD.find('button', class_='col-md-4 text-center provider-thumb-button js-shared-gallery-trigger').img['src']
        except:
            image=None

        sitter_new['image'] = [image]

    else:
        sitter_new = {}
    print(f'{len(sitters_tracking)} - {sitter_new["sitterID"]} - {sitter_new["sitterName"]}')

    return sitter_new

In [7]:
#Scroll the windows to make sure all pictures are loaded. Without this step, some informaiton may not load. 
def load_all_image(driver):
    #scroll windows from beginning to the end to load all image.
    try:
        h_win= master.get_window_size().get("height")
        h_doc= master.execute_script("return document.body.offsetHeight;")
        for w in range(h_doc//h_win):
            driver.execute_script(f"window.scrollTo(0, {w * h_win})") 
            sleep_random(3,5)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        sleep_random(3,5)
        driver.execute_script(f"window.scrollTo(0, 0)") 
        sleep_random(1,2)
    except:
        print('Hey, i scroll the document only')

In [8]:
# Parse each page then verify which sitter is not recorded then explore it.
def parse_page(soup):
    try:
        sitters=soup.findAll('div', class_='HorizontalLayout__HorizontalWrapper-sc-1b2jn93-0 gLonsc')
    except:
        sitters = []
        print('find sitters fail!')
    df_sitter = pd.read_csv('template.csv')
    for sitter in sitters:
        sitter 
        sitterID = get_sitterID(sitter) # Cannot get avatar here because of lazy loading only load 5 pictures
        # Only parse new sitter (because the postcode close together may have duplicate sitter)
        if (sitterID not in sitters_tracking) and (sitterID != None):
            sitter_new = {}
            try:
                sitterName= sitter.find('a', class_='NameRow__NameLink-t135c7-2 iLIeiF').text
                url = sitter.a['href']
            except:
                sitterName = None
                url = None
            
            try:
                city = sitter.find('span', class_="InfoColumn__Location-qduboa-2 gJTzwa").text
                province = city.split()[-2].replace(',','')
                postcode = city.split()[-1]
                city = city[0:city.find(',')]
            except:
                city = None
                province = None
                postcode = None
            
            try:
                rate_str = soup.find('div', class_='StarRating__SvgWrapper-sc-1i588cx-0 eSKMHw')['aria-label']
                rate = float(rate_str[8:rate_str.find('out')])
            except:
                rate = None
            if url != None:
                detail.get(url)
                sitter_new['sitterID'] = [sitterID]
                sitter_new['sitterName'] = [sitterName]
                sitter_new['province'] = [province]
                sitter_new['city'] = [city]
                sitter_new['postcode'] = [postcode]
                sitter_new['rate'] = [rate]
            sleep_random(1,5) #wait for detail page loading
            load_all_image(detail)
            sitters_tracking.append(sitterID)
            
            df_new = pd.DataFrame.from_dict(parse_sitter(detail, sitter_new))
            df_sitter = pd.concat([df_sitter,df_new], ignore_index=True)
    #adding new sitters (from one page) to exists file.
    df_sitter.to_csv('rover.csv', header=False, index = False, mode='a')

In [9]:
master.get("https://www.rover.com/ca")


In [10]:
#driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.COMMAND + 't')
#windows_handles = driver.window_handles

In [11]:
# Choose some options to pass the begining screen.
try:
# Choose checkbox Dog
    # Wait for checkbox loaded.
    WebDriverWait(master, 3).until(EC.presence_of_element_located((By.ID, 'element-id-2-element-id-4')))
    dog = master.find_element(By.ID, 'element-id-2-element-id-4')
    if dog.is_selected() == False:
        click_object(master, dog)
# Choose checkbox Cat
    cat = master.find_element(By.ID, 'element-id-1-element-id-5')
    if cat.is_selected() == False:
        click_object(master, cat)
# Choose House sitting (include all sitter)   
    sitter = master.find_element(By.XPATH, '//*[@id="react-search-container"]/div/div[2]/div[1]/div[2]/fieldset/div/div[2]')
    click_object(master, sitter)
    
# Click Search button   
    search_ = master.find_element(By.XPATH, '//*[@id="react-search-container"]/div/div[2]/div[3]/div[2]/button/div/span')
    click_object(master, search_)
    search_ = master.find_element(By.XPATH, '//*[@id="base-content"]/div[1]/div/div/div/div[3]/button/div/span')
    click_object(master, search_)

except:
    print ('First filter fail!')

sleep_random(3,10)
## Second filter windows
try:
    next_ = master.find_element(By.XPATH, '//*[@id="base-content"]/div[1]/div/div/div/div[3]/button/div/span')
    click_object(master, next_)
except:
    print ('Next filter fail!')
sleep_random(3,10)

## Final filter windows
try:
    search_ = master.find_element(By.XPATH, '//*[@id="base-content"]/div[1]/div/div/div/div[3]/button/div/span')
    click_object(master, search_)
except:
    print ('Final filter fail!')
    
   

In [ ]:
sitters_tracking=pd.read_csv('rover.csv')['sitterID'].tolist()
# csv file is the list of postcodes or cities. the Scrap will scan 400km around that place.
postals = pd.read_csv('postcodes.csv')['postcode'].tolist()
 

for postal in postals:
    postal_box = master.find_element(By.ID,'location-input-sidebar')
    postal_box.click()
    postal_box.clear()
    postal_box.send_keys(postal)
    sleep_random(1,2)
    postal_box.send_keys(Keys.ARROW_DOWN)
    sleep_random(1,2)
    postal_box.send_keys(Keys.RETURN) 
    sleep_random(1,3)
    zoom_out_map(master)
    cont_flag = True
    while cont_flag : #load each page then parse
        load_all_image(master)
        soup = BeautifulSoup(master.page_source, 'lxml')
        parse_page(soup) 
        #cont_flag = False # Use to test 1 page
        #Check if next page still Enable
        next_page = soup.find(class_='next') 
        if (next_page == None) or (next_page.a['aria-disabled'] == 'true'):
            cont_flag = False
        else:
            try:
                master.find_element(By.CSS_SELECTOR,'#base-content > div.HorizontalLayout__HorizontalWrapper-sc-1b2jn93-0.gitsBo.SearchPagination__SearchPaginationWrapper-u5c70y-0.fDNMSD > nav > ul > li.next > a').click()
                sleep_random(20,50) # wait for loading page master.
            except:
                print('Click next fail')
                cont_flag=False
                
# finish parse

26 - ['gXG39w2N'] - ['Jason']
27 - ['Qjb6je4Q'] - ['Alissa']
28 - ['AyeXyZPA'] - ['Brooklyn']
29 - ['gDG0mM2N'] - ['Deibourei']
30 - ['QO9Ol42N'] - ['Annie']
31 - ['gRWKVo2Q'] - ['Exelia']
32 - ['gYW6Rk6A'] - ['Mia']
33 - ['A5JPJXVN'] - ['Danielle']
34 - ['N8ZEzxqQ'] - ['Courtney']
35 - ['gRWz8pWQ'] - ['Yana']
36 - ['Nq1rq4pN'] - ['Patty']
37 - ['gLMKL8PQ'] - ['Albe']
38 - ['A5vxv6VN'] - ['Ashley']
39 - ['NEzZzYdg'] - ['Adam']
40 - ['NKRa4M9A'] - ['Ann']
41 - ['Q9zxrPkA'] - ['Kelly']
42 - ['AzZL7DRA'] - ['Ema & Ryan']
43 - ['NpYv2vvg'] - ['Mia']
44 - ['gWWOk9lg'] - ['Carianne']
45 - ['AzZ9VOMA'] - ['Suzanne']
46 - ['Nq0ObbLA'] - ['Martin']
47 - ['gmy7ml8g'] - ['Tara']
48 - ['gXek0rmQ'] - ['Sunni']
49 - ['gmKkkY6A'] - ['Nancy']
50 - ['AelGDRBQ'] - ['Krooti']
51 - ['NEzeppDg'] - ['Buse']
52 - ['N8ZEYL9Q'] - ['Siripak']
53 - ['N0pY6R6N'] - ['Sneha']
54 - ['AexwG7oQ'] - ['Lera']
55 - ['g45ajrRQ'] - ['Brent']
56 - ['QjK5a7yA'] - ['Tina']
57 - ['ZgLmWjBQ'] - ['Sarah']
58 - ['N0XVyGaN'] - ['V

283 - ['gm1BrE7g'] - ['Quyen']
284 - ['Qr1pEJGg'] - ['Alexa']
285 - ['APZEe8Kg'] - ['Rosalie']
286 - ['Q3lBPLkA'] - ['Aley']
287 - ['gxbEMZ8A'] - ['Vanessa']
288 - ['g45BoPvQ'] - ['Micha']
289 - ['gYaem8pg'] - ['Yahya']
290 - ['gVmZEXBN'] - ['Sarah']
291 - ['NEE8K3RN'] - ['Tara']
292 - ['APM0RrYN'] - ['Tara']
293 - ['gWeVxJxA'] - ['Jane']
294 - ['AMaRJdlA'] - ['Melissa']
295 - ['NvXXPbDQ'] - ['Ikinori']
296 - ['NbepjVRA'] - ['Simon']
297 - ['gYGP8kEA'] - ['Rebecca']
298 - ['NqJ5yq0g'] - ['Nicole']
299 - ['Qn56qq0g'] - ['Rolaine Roro']
300 - ['gXJOGD6A'] - ['Rachel']
301 - ['AGYXOzRg'] - ['Vandana']
302 - ['QO9nzEjN'] - ['Samantha']
303 - ['gWpjOBEQ'] - ['Joyce & Alex']
304 - ['AZebdaxQ'] - ['Aileen']
305 - ['AMPl5LOQ'] - ['Megan']
306 - ['QkjYEwbg'] - ['Sabrina']
307 - ['AMaBvPaA'] - ['Divy']
308 - ['g4Gdw40N'] - ['Samantha']
309 - ['Q1py2vKg'] - ['Andreita']
310 - ['NJBb2V9N'] - ['Johnathan']
311 - ['AolBWZLN'] - ['Katrina']
312 - ['Ao2a3M9Q'] - ['Kartik']
313 - ['APaKYBdN'] - ['Sarah

538 - ['g4a1YYoQ'] - ['Eliane']
539 - ['QnBRJv0Q'] - ['Saianvita']
540 - ['Ay6aGP3N'] - ['Cybele']
541 - ['Q3GqlYOQ'] - ['Karina']
542 - ['Nq1dVd6N'] - ['Della']
543 - ['AzdZOZJg'] - ['Ryan']
544 - ['Ae5rJ84A'] - ['Tania']
545 - ['gmd0pO1N'] - ['María']
546 - ['XNb8rERQ'] - ['James']
547 - ['gmdjXZWN'] - ['Tasha']
548 - ['Q1ly99Dg'] - ['Kelsey']
549 - ['gXrVenJA'] - ['Charlotte']
550 - ['Ao2deO7Q'] - ['Andres']
551 - ['A5J1zWEN'] - ['Paula']
552 - ['Q9J6y8MA'] - ['Priscila']
553 - ['N8GnykmA'] - ['caroline']
554 - ['AdmV0mqN'] - ['Tess']
555 - ['Nq1O31PN'] - ['Ecem Asya']
556 - ['gLMqVlMQ'] - ['Kevin']
557 - ['NEbmkBRg'] - ['Danny']
558 - ['g7Gbr0qg'] - ['Suryaprakash']
559 - ['QkY8axwN'] - ['Victoria']
560 - ['APZ783kg'] - ['Maria']
561 - ['g7mGyj3A'] - ['Michael']
562 - ['gV73xXLQ'] - ['Sofia']
563 - ['g2Veow2g'] - ['Gunjan']
564 - ['AoM2nmeg'] - ['Alice']
565 - ['N6xrMBxQ'] - ['Ashley']
566 - ['AZq6B9PQ'] - ['R']
567 - ['gRWnbWEQ'] - ['Jennifer']
568 - ['QkajPpyN'] - ['Darla']
569 -

792 - ['gYW2KmMA'] - ['Varshini']
793 - ['g7mdndwA'] - ['Montaqúe']
794 - ['gajdOpoQ'] - ['Adrian']
795 - ['Ao2aDeoQ'] - ['Eliza']
796 - ['Np1Pbrvg'] - ['Marcela']
797 - ['qNq3EWpQ'] - ['Kelly']
798 - ['gx85YXXg'] - ['Kaid']
799 - ['g4GG7p1N'] - ['Supriya']
800 - ['gLDRx01g'] - ['Patricia']
801 - ['g4pwxb0g'] - ['Abed']
802 - ['N0p331dN'] - ['Victoria']
803 - ['APadWmdN'] - ['Amanda']
804 - ['gLZ1Mp5A'] - ['Omar']
805 - ['N0J0GG6Q'] - ['Serena']
806 - ['QjaXKKyg'] - ['Cristina']
807 - ['AdojdRwQ'] - ['Matthew']
808 - ['gm1jd87g'] - ['Tamarah']
809 - ['gYpykwkN'] - ['Natasha']
810 - ['Np4n2KJN'] - ['Debbie']
811 - ['Q39xeeYN'] - ['Donna & Karen']
812 - ['gxoybLqN'] - ['Abhishek']
813 - ['gxbwq5XA'] - ['Amber']
814 - ['gWplPmzQ'] - ['Rachel']
815 - ['N8d3EX9N'] - ['parth']
816 - ['gBVJJXOg'] - ['Simeon']
817 - ['QnBBOGJQ'] - ['Gloria & Orlando']
818 - ['gDBxLDGA'] - ['Markysha']
819 - ['gLbobGjA'] - ['Tanveer']
820 - ['Q9brjOrA'] - ['Cynthia']
821 - ['N0l0V8YN'] - ['Gina']
822 - ['g2q84y

1043 - ['g45wL2ZQ'] - ['Olivia']
1044 - ['AZ1ox4vQ'] - ['Brittany']
1045 - ['NEz6lMkg'] - ['Alannah']
1046 - ['N0JErqeQ'] - ['Leslie']
1047 - ['NvdEannQ'] - ['Zlatka']
1048 - ['QnBd600Q'] - ['Sebastian']
1049 - ['AzLKeBBg'] - ['Leighanne']
1050 - ['gLMRVXPQ'] - ['Valeria']
1051 - ['Np6rGawQ'] - ['Phanie']
1052 - ['gLbnmljA'] - ['Mercedes']
1053 - ['AexJpZzQ'] - ['Cheryl']
1054 - ['AzZ64jWA'] - ['Danielle']
1055 - ['NvXJJ9RQ'] - ['Carly']
1056 - ['gab52VkQ'] - ['Olivia']
1057 - ['AGYJPrXg'] - ['Cole']
1058 - ['g4aJmM2Q'] - ['Vicky']
1059 - ['Q9zm5xnA'] - ['Sung Ching & Anna']
1060 - ['gWWmVwdg'] - ['Olivia']
1061 - ['gm1JjZMg'] - ['Tara']
1062 - ['Qn7PGMZA'] - ['Rebekah']
1063 - ['gLOLeyWN'] - ['Jessica']
1064 - ['Qr96BWxg'] - ['Sandi']
1065 - ['gYGRmdpA'] - ['Megan']
1066 - ['N0JW8orQ'] - ['Sandy']
1067 - ['Ad8RvBPA'] - ['Fiona']
1068 - ['NbnvGJnN'] - ['Kendra']
1069 - ['Np1e5Gag'] - ['Jacqueline']
1070 - ['glGXylPN'] - ['Jessica']
1071 - ['NJ2Pj59N'] - ['Emily']
1072 - ['A58Xl53Q'] - 

1290 - ['gWaPOJzA'] - ['Lauren']
1291 - ['gD4Bz72g'] - ['Mariel']
1292 - ['gxZp31vA'] - ['Caitlin']
1293 - ['gVDqm1JA'] - ['Kristen']
1294 - ['QOWM8R2Q'] - ['Shivonna']
1295 - ['NJBq6G8N'] - ['Miriam']
1296 - ['NEZXZ9og'] - ['Danielle & Danielle']
1297 - ['gBGrlzDQ'] - ['Nicole']
1298 - ['NbOV4zRg'] - ['Alex']
1299 - ['NKRzkoMA'] - ['Alannah']
1300 - ['gleVR7YQ'] - ['Emmanuel']
1301 - ['NqO0XPPQ'] - ['Nicole']
1302 - ['NpPKqyGN'] - ['Gabby']
1303 - ['g2paRwPA'] - ['Stephanie']
1304 - ['QOr1v8mg'] - ['Maria']
1305 - ['N6lJaY3Q'] - ['Mackenzie']
1306 - ['N84OO3EN'] - ['Taylor']
1307 - ['g2VOxdPg'] - ['Nick']
1308 - ['NJnmGLJN'] - ['Bohan']
1309 - ['gD8Rl63g'] - ['Antonella']
1310 - ['N84m54EN'] - ['Dave']
1311 - ['gWp4zEbQ'] - ['Barbara']
1312 - ['Q3lE6GWA'] - ['Hannah']
1313 - ['gDnYOoaA'] - ['Nicholas']
1314 - ['gXa1bVJQ'] - ['Naitik']
1315 - ['gVGy9XDQ'] - ['Tafean']
1316 - ['NJMBvEdA'] - ['Shannon']
1317 - ['AMDp2a1Q'] - ['Krista']
1318 - ['g2p5rJRA'] - ['Gabrielle']
1319 - ['gYamGor

1537 - ['N0bGlb0Q'] - ['Weronika']
1538 - ['QOWMrn9Q'] - ['Brandi Lynn']
1539 - ['gmJq6l8Q'] - ['Christine']
1540 - ['gBE0y88N'] - ['Brittany']
1541 - ['gmyxajVg'] - ['Aby Saju']
1542 - ['N0XJwXdN'] - ['Alysha']
1543 - ['QkajdDpN'] - ['Erika']
1544 - ['NbnYRdnN'] - ['Alexa']
1545 - ['gaxyp0oN'] - ['Stephanie']
1546 - ['AzPZpWKg'] - ['Grace']
1547 - ['jgV9yjDN'] - ['Kevin']
1548 - ['AzoM1lRN'] - ['AAliyah']
1549 - ['gmyPKoBg'] - ['Lori-Ann']
1550 - ['A5GaMDZg'] - ['Jess']
1551 - ['NEZj0aMg'] - ['Myt']
1552 - ['Q3GDy80Q'] - ['Kathleen']
1553 - ['Ay6GxP3N'] - ['Susan']
1554 - ['AzZ4WPKA'] - ['Sonja']
1555 - ['g2qOZL4N'] - ['Emily']
1556 - ['gajnaqOQ'] - ['Jackie']
1557 - ['AyLjd8PQ'] - ['Ellen']
1558 - ['g2qjxk2N'] - ['Parshwa']
1559 - ['gmd47VMN'] - ['Andrea']
1560 - ['gD8Dx4ag'] - ['Silvana']
1561 - ['g24nyBpA'] - ['Mariah']
1562 - ['NvJlW7oN'] - ['Abbey']
1563 - ['gle1ejEQ'] - ['Melanie']
1564 - ['QnvJ5V0Q'] - ['Erica']
1565 - ['gRWlwjDQ'] - ['Gretel']
1566 - ['gV75pVOQ'] - ['Jenna']
1

1785 - ['bN8nDBeQ'] - ['Melanie']
1786 - ['gYGX9o6A'] - ['Alyssa']
1787 - ['8gRr3J2g'] - ['Cara']
1788 - ['eQ94YqGg'] - ['Deliann']
1789 - ['AZ1RBvlQ'] - ['Morgane']
1790 - ['gm1MGrPg'] - ['Caitlin & Trenton']
1791 - ['gBqrm0jN'] - ['Cindy']
1792 - ['QkYwDLdN'] - ['Ashley']
1793 - ['Np6yZMJQ'] - ['Cheyenne']
1794 - ['gBGo6jjQ'] - ['Sofia']
1795 - ['gaj8eGoQ'] - ['Mary']
1796 - ['NKReaGeA'] - ['Beverly']
1797 - ['gYdVLz5N'] - ['Erin']
1798 - ['eQ9VJm7A'] - ['Julia']
1799 - ['Qn5KKXpg'] - ['Amber']
1800 - ['Np4a98vN'] - ['Natividad']
1801 - ['gXeGyRJQ'] - ['Hannah']
1802 - ['NqOb6zXQ'] - ['James']
1803 - ['gle498PQ'] - ['Victoria']
1804 - ['gmdjKXWN'] - ['Cheyenne']
1805 - ['QjVxmX2Q'] - ['Fabu']
1806 - ['NbP3v83Q'] - ['Savannah']
1807 - ['AyebMYRA'] - ['Dameon']
1808 - ['gXJK47pA'] - ['Micayla']
1809 - ['Ad9vJJ8N'] - ['Haley']
1810 - ['gVaJ2zON'] - ['Kay']
1811 - ['gaxvzBRN'] - ['Suzanna']
1812 - ['Ay6O8JPN'] - ['Cailyn']
1813 - ['QkYE1beN'] - ['Shelby Marie']
1814 - ['NbO8lwrg'] - ['Me

2031 - ['NqOM8MLQ'] - ['Claire']
2032 - ['gYpMOEMN'] - ['Morgan']
2033 - ['glv0Pq5A'] - ['Terry']
2034 - ['gWeLqwEA'] - ['Keira']
2035 - ['Q3lmBl0A'] - ['Bethany']
2036 - ['gRK0e2VQ'] - ['Nesreen']
2037 - ['AGOZKK6A'] - ['Lori']
2038 - ['AP5ywZ5N'] - ['Owen']
2039 - ['A58eRvjQ'] - ['Kyrie']
2040 - ['gwyoXj6A'] - ['Helen']
2041 - ['Qjbb914Q'] - ['Isabel']
2042 - ['AZ1646vQ'] - ['Karly']
2043 - ['QrdoyX8N'] - ['Jasandeep']
2044 - ['g4p04d2g'] - ['Lauren']
2045 - ['NKYq6r9Q'] - ['Jes']
2046 - ['Q1lj4wog'] - ['Lucie']
2047 - ['Q3lo770A'] - ['Katy']
2048 - ['g2lyEBZA'] - ['Jacinta']
2049 - ['QkPwKmyA'] - ['Yasmine']
2050 - ['QjblvdkQ'] - ['Hannah']
2051 - ['AMa2wD4A'] - ['Norma']
2052 - ['AP55V55N'] - ['Chad']
2053 - ['AyLE9GqQ'] - ['Cerena']
2054 - ['NJB4z4pN'] - ['Ashley']
2055 - ['NvXvlxRQ'] - ['Aaron']
2056 - ['A5GvLbjg'] - ['Kara']
2057 - ['NEE80zkN'] - ['Jaemie']
2058 - ['gxd7W9mg'] - ['Rosario']
2059 - ['NvJZbWJN'] - ['Claire']
2060 - ['gYWBne6A'] - ['Zachary']
2061 - ['gVmqXeJN'] - 

2279 - ['Q9zqezPA'] - ['Hila']
2280 - ['NKY9a7EQ'] - ['Jenna']
2281 - ['Q9KDMzGN'] - ['Elisabeth']
2282 - ['gmKpn6MA'] - ['Max']
2283 - ['A5vbwzoN'] - ['Taylin']
2284 - ['Q9JkdV7A'] - ['Chrissy']
2285 - ['gBeERBEN'] - ['Naya']
2286 - ['gmdY4ERN'] - ['Sophia']
2287 - ['g4qr5EMQ'] - ['Cass']
2288 - ['gLD4ROjg'] - ['Grace']
2289 - ['gVaoLwJN'] - ['Maya']
2290 - ['APaPa4XN'] - ['Kara']
2291 - ['Ao27pBoQ'] - ['Karen']
2292 - ['N6D4knxQ'] - ['Piper']
2293 - ['g7G8D7xg'] - ['Ivy']
2294 - ['AZqLL9PQ'] - ['Jacqueline']
2295 - ['g2lqVPPA'] - ['Rae']
2296 - ['QkjavDpg'] - ['Alexandra']
2297 - ['Q39k66kN'] - ['Nicole']
2298 - ['gRKkrdPQ'] - ['Emmy']
2299 - ['N6GOEExg'] - ['Faith']
2300 - ['NEz2xMkg'] - ['Kaitlyn']
2301 - ['gXeD22qQ'] - ['Megan']
2302 - ['gx8q2rxg'] - ['Chelsea']
2303 - ['AeRoELoN'] - ['Gwyneth']
2304 - ['AMR9XbmN'] - ['Jenn']
2305 - ['gwZJpd9N'] - ['Brayden']
2306 - ['yglK62lN'] - ['Talitha']
2307 - ['VgxRne7A'] - ['Kaila']
2308 - ['VQnz28qQ'] - ['Thomas']
2309 - ['QjwwDLkA'] - ['